In [ ]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Sequential, models, layers
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
path = "../../00.Data/07.ML/data/Face Mask Dataset/"

dataset = {
    "image_path" : [],
    "mask_status" : [],
    "where" : []
}

for where in os.listdir(path) :
    for status in os.listdir(path+"/"+where) :
        for image in glob.glob(path+where+"/"+status+"/"+"*.png") :
            dataset["image_path"].append(image)
            dataset["mask_status"].append(status)
            dataset["where"].append(where)

dataset = pd.DataFrame(dataset)
dataset.head()

In [ ]:
print(f"With Mask:", dataset.value_counts("mask_status")[0])
print(f"Without Mask:", dataset.value_counts("mask_status")[1])

sns.countplot(x = dataset["mask_status"])

In [ ]:
plt.figure(figsize = (15,10))
for i in range(9) :
  random = np.random.randint(1, len(dataset))
  plt.subplot(3, 3, i+1)
  plt.imshow(cv2.imread(dataset.loc[random, "image_path"]))
  plt.title(dataset.loc[random, "mask_status"], size=15)
  plt.xticks([])
  plt.yticks([])
plt.show()

In [ ]:
train_df = dataset[dataset["where"] == "Train"]
test_df = dataset[dataset["where"] == "Test"]
valid_df = dataset[dataset["where"] == "Validation"]
print(train_df.head(10))

In [ ]:
plt.figure(figsize = (15,10))
plt.subplot(1, 3, 1)
sns.countplot(x = train_df["mask_status"])
plt.title("Training Dataset", size=10)
plt.subplot(1, 3, 2)
sns.countplot(x = test_df["mask_status"])
plt.title("Test Dataset", size=10)
plt.subplot(1, 3, 3)
sns.countplot(x = valid_df["mask_status"])
plt.title("Validation Dataset", size=10)
plt.show()

In [ ]:
train_df = train_df.reset_index().drop("index", axis=1)
data = []
image_size = 150

for i in range(len(train_df)) :
    img_array = cv2.imread(train_df["image_path"][i], cv2.IMREAD_GRAYSCALE)
    new_image_array = cv2.resize(img_array, (image_size, image_size))

    if train_df["mask_status"][i] == "WithMask" :
        data.append([new_image_array, 1])
    else :
        data.append([new_image_array, 0])

In [ ]:
#data = np.array(data)
np.random.shuffle(data)
type(data)

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(10, 6))

for row in range(2) :
    for col in range(3) :
        image_index = row*100+col

        ax[row, col].axis("off")
        ax[row, col].imshow(data[image_index][0], cmap = 'gray')

    if data[image_index][1] == 0 :
        ax[row, col].set_title("Without Mask")
    else :
        ax[row, col].set_title("with Mask")

plt.show()

In [ ]:
x = []
y = []

for image in data :
    x.append(image[0])
    y.append(image[1])

x = np.array(x)
y = np.array(y)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=50)

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(5,5),strides=(1,1), padding='same', activation='relu', input_shape=(150,150,1)),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    layers.Conv2D(64, (2,2), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(1000, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model.summary()

In [ ]:
import time

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy'])

x_train = x_train.reshape(len(x_train), x_train.shape[1], x_train.shape[2], 1)
x_val = x_val.reshape(len(x_val), x_val.shape[1], x_val.shape[2], 1)

history = model.fit(x_train, y_train, epochs=5, batch_size=32)

In [ ]:
prediction = (model.predict(x_val) > 0.5).astype("int32")
print(classification_report(y_val, prediction))
print(confusion_matrix(y_val, prediction))